# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

### Importer les données

In [90]:
df_categories = pd.read_json("./data/US_category_id.json")
df_categories["id"] = df_categories["items"].apply(lambda x : int(x.get("id")))

df_o = df_youtube.merge(df_categories, right_on = 'id', left_on = "category_id")
df_o.rename({"video_id": "_id"}, axis=1, inplace=True)

In [91]:
df_o_w_d = df_o.drop_duplicates("_id")

In [92]:
df_o._id.shape, df_o._id.unique().shape

((7992,), (2364,))

In [93]:
collection.insert_many(df_o_w_d.to_dict(orient = 'records'))
collection.find_one()

{'_id': 'XpVt6Z1Gjjo',
 'category_id': 24,
 'channel_title': 'Logan Paul Vlogs',
 'comment_total': 46245,
 'date': 13.09,
 'dislikes': 5931,
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'id': 24,
 'items': {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"',
  'id': '24',
  'kind': 'youtube#videoCategory',
  'snippet': {'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
   'title': 'Entertainment'}},
 'kind': 'youtube#videoCategoryListResponse',
 'likes': 320053,
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'title': '1 YEAR OF VLOGGING --

## Question 1  

In [94]:
import pprint

In [95]:
for video in collection.find({"channel_title" : "Apple"}):
    pprint.pprint(video)

{'_id': 'K4wEI5zhHB0',
 'category_id': 28,
 'channel_title': 'Apple',
 'comment_total': 0,
 'date': 13.09,
 'dislikes': 26679,
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'id': 28,
 'items': {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"',
           'id': '28',
           'kind': 'youtube#videoCategory',
           'snippet': {'assignable': True,
                       'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                       'title': 'Science & Technology'}},
 'kind': 'youtube#videoCategoryListResponse',
 'likes': 185853,
 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 '
         'Bionic|augmented reality|emoji|animoji|Face ID|Apple '
         'Pay|camera|smartphone',
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'title': 'iPhone X — Introducing iPhone X — Apple',
 'views': 7860119}
{'_id': 'A9k88sMyiJM',
 'category_id': 28,
 'channel_title': 'Apple',
 'comment_total': 0,
 'date': 18.09,
 

## Question 2

In [96]:
for a in collection.aggregate([{"$group" : { "_id" : "$category_id", "videosNumberByCategory" : { "$sum" : 1}}}]):
    pprint.pprint(a)

{'_id': 19, 'videosNumberByCategory': 18}
{'_id': 29, 'videosNumberByCategory': 5}
{'_id': 15, 'videosNumberByCategory': 37}
{'_id': 20, 'videosNumberByCategory': 29}
{'_id': 23, 'videosNumberByCategory': 213}
{'_id': 22, 'videosNumberByCategory': 257}
{'_id': 25, 'videosNumberByCategory': 198}
{'_id': 17, 'videosNumberByCategory': 149}
{'_id': 2, 'videosNumberByCategory': 37}
{'_id': 26, 'videosNumberByCategory': 262}
{'_id': 27, 'videosNumberByCategory': 94}
{'_id': 43, 'videosNumberByCategory': 1}
{'_id': 1, 'videosNumberByCategory': 101}
{'_id': 24, 'videosNumberByCategory': 477}
{'_id': 28, 'videosNumberByCategory': 151}
{'_id': 10, 'videosNumberByCategory': 335}


## Question 3

In [206]:
df_o_w_d["tags"] = df_o_w_d["tags"].apply(lambda x : x.split('|'))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [87]:
df_o_w_d

,_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,etag,items,kind,id
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,"[mtv, video, online, official, tv, television,...",274358,9215,477,838,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,24,"[America's Got Talent 2017, america's got tale...",473691,14740,415,1696,https://i.ytimg.com/vi/vu_9muoxT50/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,24,"[panda, what should we eat, buzzfeed, comedy, ...",282858,14870,300,1398,https://i.ytimg.com/vi/ZQK1F0wz6z4/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,24,"[TMZ2016FS11221, TMZ, Hollywood, Celebrity, En...",703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
5,4Yue-q9Jdbk,SUPERFRUIT REACTS TO TEENS REACT TO SUPERFRUIT,SUPERFRUIT,24,"[superfruit, super, fruit, scott, hoying, mitc...",255967,21817,293,2017,https://i.ytimg.com/vi/4Yue-q9Jdbk/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
6,1ZNZY-gd3K0,Joe Arpaio: Last Week Tonight with John Oliver...,LastWeekTonight,24,"[last week tonight joe arpaio, joe arpaio, joh...",3298456,65730,4360,9504,https://i.ytimg.com/vi/1ZNZY-gd3K0/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
7,JO7X9ZPoAp8,Blind People Describe Beauty,WatchCut Video,24,"[blind people, buzzfeed, beauty, beautiful, bl...",136643,6218,127,524,https://i.ytimg.com/vi/JO7X9ZPoAp8/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
8,oDIDZ9EmQfA,Jumping Through Impossible Shapes!,Rclbeauty101,24,"[Jumping Through impossible shapes, rclbeauty1...",796896,39791,1252,3868,https://i.ytimg.com/vi/oDIDZ9EmQfA/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24
9,zAtHxJvSczA,Does Maggie Gyllenhaal Have Taylor Swift’s Sca...,Watch What Happens Live with Andy Cohen,24,"[What What Happens live, reality, interview, f...",2932,71,3,15,https://i.ytimg.com/vi/zAtHxJvSczA/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",youtube#videoCategoryListResponse,24


In [200]:
collection.update_many({}, {'$set': {'tags': df_o_w_d.to_dict(orient = 'list')}})
collection.find_one()

TypeError: to_dict() got an unexpected keyword argument 'orient'

In [216]:
for r, row in df_o_w_d.iterrows(): 
    collection.update_one({"_id":row._id}, {"$set":{'tags':row.tags}})

In [215]:
collection.find_one()

{'_id': 'XpVt6Z1Gjjo',
 'category_id': 24,
 'channel_title': 'Logan Paul Vlogs',
 'comment_total': 46245,
 'date': 13.09,
 'dislikes': 5931,
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'id': 24,
 'items': {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"',
  'id': '24',
  'kind': 'youtube#videoCategory',
  'snippet': {'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
   'title': 'Entertainment'}},
 'kind': 'youtube#videoCategoryListResponse',
 'likes': 320053,
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond butto

## Question 4

In [125]:
for video in collection.find({}).sort("views", pymongo.DESCENDING).limit(5):
    pprint.pprint(video)

{'_id': 'tt2k8PGm-TI',
 'category_id': 10,
 'channel_title': 'ZaynVEVO',
 'comment_total': 100661,
 'date': 13.09,
 'dislikes': 28049,
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'id': 10,
 'items': {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
           'id': '10',
           'kind': 'youtube#videoCategory',
           'snippet': {'assignable': True,
                       'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
                       'title': 'Music'}},
 'kind': 'youtube#videoCategoryListResponse',
 'likes': 1431683,
 'tags': 'Dusk Till Dawn|Pop|RCA Records Label|ZAYN feat. Sia',
 'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
 'title': 'ZAYN - Dusk Till Dawn ft. Sia',
 'views': 36323498}
{'_id': 'MBdVXkSdhwU',
 'category_id': 10,
 'channel_title': 'ibighit',
 'comment_total': 501435,
 'date': 19.09,
 'dislikes': 46477,
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'id': 10,
 'items': {

## Question 5

In [177]:
for a in collection.aggregate([{"$group" : { "_id" : "$category_id", "videosMeanViewByCategory" : { "$avg" : '$views'}}}]):
    pprint.pprint(a)

{'_id': 19, 'videosMeanViewByCategory': 300664.1111111111}
{'_id': 29, 'videosMeanViewByCategory': 572023.2}
{'_id': 15, 'videosMeanViewByCategory': 412381.6216216216}
{'_id': 20, 'videosMeanViewByCategory': 492390.2413793103}
{'_id': 23, 'videosMeanViewByCategory': 828485.2676056338}
{'_id': 22, 'videosMeanViewByCategory': 597891.1789883268}
{'_id': 25, 'videosMeanViewByCategory': 374883.0505050505}
{'_id': 17, 'videosMeanViewByCategory': 470686.1677852349}
{'_id': 2, 'videosMeanViewByCategory': 415129.0540540541}
{'_id': 26, 'videosMeanViewByCategory': 405748.2900763359}
{'_id': 27, 'videosMeanViewByCategory': 368229.0425531915}
{'_id': 43, 'videosMeanViewByCategory': 8259.0}
{'_id': 1, 'videosMeanViewByCategory': 558289.1881188119}
{'_id': 24, 'videosMeanViewByCategory': 555071.1740041929}
{'_id': 28, 'videosMeanViewByCategory': 515854.13907284767}
{'_id': 10, 'videosMeanViewByCategory': 697598.5910447761}


## Question 6 

In [190]:
for a in collection.aggregate([{"$group" : { "_id" : "$channel_title", "meanLikes" : { "$avg" : '$likes'}}}, { '$sort' : { 'meanLikes' : -1}}, { '$limit' : 5 }]):
    pprint.pprint(a)

{'_id': 'ZaynVEVO', 'meanLikes': 1431683.0}
{'_id': 'ibighit', 'meanLikes': 1112071.0}
{'_id': 'melanie martinez', 'meanLikes': 902166.0}
{'_id': 'jypentertainment', 'meanLikes': 721796.0}
{'_id': 'Logan Paul Vlogs', 'meanLikes': 601533.5}
